# Consumo Calculado y Predicho

## A considerar:
Interpretación de los Resultados:

- Valores Positivos: Indican que se espera tener un inventario positivo de ese insumo al final del mes. Por ejemplo, para mayo de 2024, se espera tener un inventario de aproximadamente 19,384 unidades.
- Valores Negativos: Indican que se espera un inventario negativo, lo que implica una posible escasez o déficit del insumo. Por ejemplo, para septiembre de 2024, se predice un déficit de aproximadamente 28,569 unidades.
  
Identificación de la Fecha y Cantidad Óptima para Pedido:

- Dado que la fecha óptima para hacer un pedido es aquella en la que el inventario proyectado es mínimo (especialmente si es negativo), puedes tomar la fecha con el inventario más negativo como la señal para realizar un pedido.

**La cantidad a pedir debe cubrir el déficit y adicionalmente asegurar un inventario suficiente para el próximo periodo.**

## Data

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib  # Importar joblib para la serialización del modelo
import warnings

# Definir las rutas de los archivos Excel
file_paths = [
    'datasets_xlsx/2022/cuadro-insumos-medico-quirurgico-enero-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-insumos-medico-quirurgico-febrero-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-insumo-medico-quirurgico-marzo-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-medicamentos-abril-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-insumo-medico-quirurgico-de-mayo-2022.xlsx',
    'datasets_xlsx/2022/inventario-insumos-medico-quirurgico-junio-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-medico-quirurgico-julio-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-insumos-medico-quirurgico-agosto-2022.xlsx',
    'datasets_xlsx/2022/inventario-de-insumos-medico-quirurgico-septiembre-2022.xlsx',
    'datasets_xlsx/2022/inventario-insumos-medico-quirurgico-octubre-2022.xlsx',
    'datasets_xlsx/2022/inventario-medico-quirurgico-nov.-2022.xlsx',
    'datasets_xlsx/2022/inventario-medico-quirugico-dic.-2022.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-insumos-medico-quirurgico-enero-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-insumos-medico-quirurgico-febrero-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-insumos-medico-quirurgico-marzo-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-insumos-medico-quirurgico-abril-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-de-medico-quirurgico-de-mayo-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-de-medico-quirurgico-de-junio-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-de-medico-quirurgico-de-julio-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-medico-quirurgico-agosto-2023.xlsx',
    'datasets_xlsx/2023/inventario-mq-a-septiembre-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-medico-quirurgico-octubre-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-medico-quirurgico-nov.-2023.xlsx',
    'datasets_xlsx/2023/cuadro-de-inventario-medico-quirurgico-dic.-2023.xlsx',
    'datasets_xlsx/2024/cuadro-de-inventario-insumos-medico-quirurgico-enero-2024.xlsx',
    'datasets_xlsx/2024/cuadro-de-inventario-de-insumos-medicos-quirurgico-feb.-2024.xlsx',
    'datasets_xlsx/2024/inventario-insumos-medico-quirurgico-marzo-2024.xlsx',
    'datasets_xlsx/2024/cuadro-de-inventario-mq-abril-2024.xlsx'
]

months = [
    '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', 
    '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01',
    '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', 
    '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01',
    '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01'
]

# Leer los archivos Excel en DataFrames y añadir una columna para el mes correspondiente
dfs = []

for file_path, month in zip(file_paths, months):
    df = pd.read_excel(file_path)
    df = df.iloc[:, [0, 1, -2]]  # Seleccionar las columnas por índice
    df['Fecha'] = month  # Añadir la columna de la fecha
    df = df.rename(columns={df.columns[2]: 'Inventario'})  # Renombrar la columna -2
    dfs.append(df)

# Concatenar todos los DataFrames
df_combined = pd.concat(dfs, ignore_index=True)

# Renombrar las columnas
df_combined.columns = ['Codigo', 'Insumo', 'Inventario', 'Fecha']

# Eliminar filas con NaN en la columna 'Codigo'
df_combined = df_combined.dropna(subset=['Codigo', 'Insumo'])

# Obtener Dataframe Codigo, nombre
df_insumos = df_combined[['Codigo', 'Insumo']]

# Reordenar las columnas
df_combined = df_combined[['Fecha', 'Codigo', 'Insumo', 'Inventario']]

# Contar el número de ceros por cada 'Insumo'
def count_zeros(df):
    return (df['Inventario'] == 0).sum()

# Filtrar los insumos con al menos 2 ceros
filtered_insumos = df_combined.groupby('Insumo').filter(lambda x: count_zeros(x) <= 2)

# Crear una tabla pivote
pivot_table = filtered_insumos.pivot_table(
    index='Insumo',         # Las filas serán los insumos
    columns='Fecha',        # Las columnas serán las fechas
    values='Inventario',    # Los valores serán los inventarios
    aggfunc='sum'           # La función de agregación será la suma
).fillna(0)

# Contar los ceros en cada fila
zero_counts = (pivot_table == 0).sum(axis=1)

# Filtrar las filas que tienen a lo sumo 2 ceros
filtered_pivot_table = pivot_table[zero_counts <= 2]

# Calcular el consumo (diferencia de inventarios entre meses consecutivos)
consumption = filtered_pivot_table.diff(axis=1).fillna(0)

In [6]:
consumption

Fecha,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01
Insumo,,,,,,,,,,,,,,,,,,,,,
APOSITO OCULAR ADULTO ESTERIL,0.0,-15970.0,-37130.0,-73090.0,90400.0,-4550.0,-9200.0,-20500.0,-39400.0,45650.0,...,-89200.0,-20025.0,-12900.0,12900.0,0.0,6450.0,-12900.0,0.0,-1350.0,-13200.0
BANDAS ELÁSTICAS PARA FORTALECIMIENTO (SE SOLICITA -SUPER FUERTE),0.0,-15.0,-21.0,-45.0,518.0,-20.0,87.0,-402.0,-122.0,495.0,...,-19.0,-34.0,-38.0,8.0,0.0,7.0,-14.0,0.0,7.0,0.0
BANDAS ELÁSTICAS PARA FORTALECIMIENTO (SE SOLICITA FUERTE),0.0,-218.0,30.0,-62.0,256.0,-24.0,713.0,-183.0,-508.0,762.0,...,-276.0,-28.0,-251.0,235.0,-4.0,-62.0,-319.0,-18.0,7.0,-64.0
BANDAS ELÁSTICAS PARA FORTALECIMIENTO. (SE SOLICITA SUAVE),0.0,-255.0,-18.0,-54.0,70.0,-2.0,0.0,307.0,-250.0,307.0,...,-72.0,-18.0,-30.0,15.0,0.0,9.0,-18.0,-4.0,-1.0,-2.0
BOLSA COLECTORA DE ORINA DE 30 A 35 ONZA DE CAPACIDAD (900CC A 1050CC) DE PIERNA .,0.0,-9326.0,-45808.0,-40162.0,33744.0,-11920.0,-4364.0,-10460.0,-6520.0,38.0,...,-3188.0,-692.0,-172.0,164638.0,-21528.0,4514.0,-61254.0,-17618.0,15522.0,178.0
BOLSA INFUSORA A PRESION .SE SOLICITA BOLSA DE 1000cc DE CAPACIDAD,0.0,0.0,-165.0,-495.0,420.0,0.0,0.0,-105.0,-210.0,105.0,...,-85.0,0.0,-170.0,170.0,0.0,85.0,-170.0,0.0,85.0,0.0
BRAZALETE DE IDENTIFICACIÓN PARA ADULTOS. (SE SOLICITA: 12 PULGADAS DE LONGITUD),0.0,-347400.0,-2130000.0,-6067000.0,7498600.0,-103600.0,-34900.0,-1883000.0,-3673600.0,5404500.0,...,-1602100.0,-45500.0,-2966800.0,2936860.0,4700.0,-15500.0,-1638740.0,-2547352.0,97444.0,2248032.0
BRAZALETE DE IDENTIFICACIÓN PEDIATRICA. SE SOLICITA: 8 PULGADAS DE LONGITUD,0.0,-176000.0,-387300.0,-1315900.0,2225200.0,-11500.0,-28200.0,-542700.0,-1095200.0,1616800.0,...,-524400.0,-10300.0,-1070200.0,846500.0,-21900.0,7800.0,-459000.0,-768600.0,-1585.0,725285.0
CANULA NASOFARINGEA DE 20 FR A 26 FR. (SE SOLOCITA TAMAÑ0 26FR),0.0,-44400.0,-700.0,-2100.0,2800.0,0.0,0.0,-700.0,-1400.0,2100.0,...,-3170.0,0.0,-6340.0,3500.0,0.0,-20.0,-2500.0,0.0,2440.0,0.0


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib  # Importar joblib para la serialización del modelo
import json
import os
import warnings
# Entrenar y guardar los modelos SARIMAX
modelos = {}
predecibles = []
cantidad_adicional = 5000
avisos = []

output_dir = "modelos_sarimax"
os.makedirs(output_dir, exist_ok=True)

# Entrenar y guardar los modelos SARIMAX (continuación)
for insumo in consumption.index:
    try:
        df_insumo = consumption.loc[insumo].dropna()

        if len(df_insumo) < 24:
            continue

        train_end = '2024-01-31'
        train_data = df_insumo[:train_end]
        test_data = df_insumo[train_end:]

        model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        results = model.fit(disp=False)

        codigo_producto = int(df_insumos[df_insumos['Insumo'] == insumo]['Codigo'].values[0])
        joblib.dump(results, f"{output_dir}/modelo_{codigo_producto}.pkl")

        predecibles.append({
            "Codigo": int(codigo_producto),  # Convertir a entero
            "Nombre": insumo,
            "ultimos_valores": [int(x) for x in df_insumo.tail(6)]  # Convertir a enteros
        })

        pred = results.get_forecast(steps=12)
        predicted_values = pred.predicted_mean

        for fecha, inventario in predicted_values.items():
            if inventario < 0:
                # Calcular el último día del mes anterior
                mes_anterior = (pd.to_datetime(fecha) - pd.DateOffset(months=1)).strftime('%Y-%m-%d')
                mes_anterior = pd.to_datetime(mes_anterior) + pd.offsets.MonthEnd(0)
                
                cantidad_a_pedir = int(abs(inventario) + cantidad_adicional)  # Convertir a entero

                # Calcular el último día del mes actual
                fecha_deficit = pd.to_datetime(fecha) + pd.offsets.MonthEnd(0)
                
                avisos.append({
                    "codigo": int(codigo_producto),
                    "insumo": insumo,
                    "cuando pedir": mes_anterior.strftime('%Y-%m-%d'),
                    "cantidad_a_pedir": cantidad_a_pedir,
                    "fecha_a_cubrir": fecha_deficit.strftime('%Y-%m-%d')
                })
    except Exception as e:
        print(f"Error al procesar el insumo {insumo}: {e}")

C:\Users\kevin\anaconda3\envs\Feria\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\kevin\anaconda3\envs\Feria\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\kevin\anaconda3\envs\Feria\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\kevin\anaconda3\envs\Feria\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\kevin\anaconda3\envs\Feria\lib\site

In [29]:
# Guardar los JSON
with open('predecibles.json', 'w') as predecibles_file:
    json.dump(predecibles, predecibles_file, indent=4)

with open('requerimientos.json', 'w') as avisos_file:
    json.dump(avisos, avisos_file, indent=4)

print("Modelos SARIMAX guardados y archivos JSON generados con éxito.")

Modelos SARIMAX guardados y archivos JSON generados con éxito.
